# Calculating NigthLight Density (MODIS and VIIRS)

In [ ]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

## Loading, Clipping, and Masking Imagery


In [ ]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Imagery of Nighttime Lights (Harmonized DMSP-OLS and VIIRS)
dmsp = ee.ImageCollection("projects/sat-io/open-datasets/Harmonized_NTL/dmsp")
viirs = ee.ImageCollection("projects/sat-io/open-datasets/Harmonized_NTL/viirs")

# Clip the NL images to Colombia's boundaries
dmsp_COL = dmsp.map(lambda image: image.clip(colombiaMpios))
viirs_COL = viirs.map(lambda image: image.clip(colombiaMpios))

In [22]:
# # Mask function - paper suggests a DN threshold of 7. "A harmonized global nighttime light dataset 1992–2018"
# def mask_dn_greater_than_7(image):
#     b1 = image.select('b1') # band_names = first_dmsp.bandNames().getInfo()
#     mask = b1.gt(7)  # Create a mask for values > 7
#     return image.updateMask(mask)

# # Apply the mask to each image in the collections
# dmsp_masked = dmsp_COL.map(mask_dn_greater_than_7)
# viirs_masked = viirs_COL.map(mask_dn_greater_than_7)

## Calculating Average NL intensity fo DMSP (2000-2013)

In [23]:
# Defining the function to calculate the mean DN value for each municipality in a given year
def calculate_nl_mean(feature, year):

    # Selecting year
    dmsp_year = dmsp_COL.filterDate(f'{year}-01-01', f'{year}-12-31').mean()

    # Calculating the mean DN value for the selected year 
    mean_dn = dmsp_year.reduceRegion (
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('b1')

    # Dynamically name the b1 property
    mean_dn_property_name = f'nl{year}'

    # Return a new feature with only the 'DN' and 'codmpio' properties
    return ee.Feature(None, {
        mean_dn_property_name: mean_dn,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
year_results_dict = {}

for year in range(2000, 2013):

    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_nl_mean(feature, year))
    year_results_dict[f'nl_year_results{year}'] = current_year_results

    # Optionally print the first feature's info to check
    print(year_results_dict[f'nl_year_results{year}'].first().getInfo())


{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2000': 6.197840730418748}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2001': 6.504423693606144}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2002': 7.36862152231}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2003': 10.843443683714478}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2004': 7.0129664017830065}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2005': 6.248395627482799}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2006': 6.201859851589564}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio'

## Calculating Average NL intensity from VIIRS (2013-2021)

In [24]:
# Defining the function to calculate the mean DN value for each municipality in a given year
def calculate_nl_mean(feature, year):

    # Selecting year
    viirs_year = viirs_COL.filterDate(f'{year}-01-01', f'{year}-12-31').mean()

    # Calculating the mean DN value for the selected year 
    mean_dn = viirs_year.reduceRegion (
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('b1')

    # Dynamically name the b1 property
    mean_dn_property_name = f'nl{year}'

    # Return a new feature with only the 'DN' and 'codmpio' properties
    return ee.Feature(None, {
        mean_dn_property_name: mean_dn,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

for year in range(2013, 2021):

    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_nl_mean(feature, year))
    year_results_dict[f'nl_year_results{year}'] = current_year_results

    # Optionally print the first feature's info to check
    print(year_results_dict[f'nl_year_results{year}'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2013': 6.757315023330018}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2014': 6.675836197815239}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2015': 6.806950119426527}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2016': 6.680289865189956}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2017': 6.723684405516946}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2018': 6.655522775797124}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'nl2019': 7.35385678564962}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio

## Exporting CVS files to my Google Drive  
### NOTE: see the progress of exports at [GEE Task Manager](https://code.earthengine.google.com/tasks)

In [ ]:
# Define a function to export FeatureCollections to Google Drive
def export_to_drive(feature_collection, year, description_prefix="NightLightData", folder_name="EarthEngineExports"):
    # Create the export task
    export_task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=f"{description_prefix}_Year{year}",
        folder="EarthEngineExports",
        fileNamePrefix=f"NightLight_Year{year}",
        fileFormat="CSV"
    )
    # Start the export task
    export_task.start()
    print(f"Exporting {description_prefix}_Year{year} to Google Drive...")

# Loop through each item in the dictionary and initiate an export
for year, feature_collection in year_results_dict.items():
    # Extract the year from the key (which is a string like 'year_results11')
    extracted_year = int(year.replace('nl_year_results', ''))
    export_to_drive(feature_collection, extracted_year)
    
#END

Exporting NightLightData_Year2000 to Google Drive...
Exporting NightLightData_Year2001 to Google Drive...
Exporting NightLightData_Year2002 to Google Drive...
Exporting NightLightData_Year2003 to Google Drive...
Exporting NightLightData_Year2004 to Google Drive...
Exporting NightLightData_Year2005 to Google Drive...
Exporting NightLightData_Year2006 to Google Drive...
Exporting NightLightData_Year2007 to Google Drive...
Exporting NightLightData_Year2008 to Google Drive...
Exporting NightLightData_Year2009 to Google Drive...
Exporting NightLightData_Year2010 to Google Drive...
Exporting NightLightData_Year2011 to Google Drive...
Exporting NightLightData_Year2012 to Google Drive...
Exporting NightLightData_Year2013 to Google Drive...
Exporting NightLightData_Year2014 to Google Drive...
Exporting NightLightData_Year2015 to Google Drive...
Exporting NightLightData_Year2016 to Google Drive...
Exporting NightLightData_Year2017 to Google Drive...
Exporting NightLightData_Year2018 to Google Dr